In [1]:
# Utility
import re
import numpy as np
import pandas as pd

# OS I/O
import requests
import inspect
import sys
import os

# Image Processing
import cv2

currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir) 

from Utils._fe_utils import download_video_ffmpeg

In [2]:
print(download_video_ffmpeg("https://aistdancedb.ongaaccel.jp/v1.0.0/video/2M/gPO_sBM_c01_d10_mPO0_ch04.mp4"))

./gPO_sBM_c01_d10_mPO0_ch04.mp4


In [ ]:
# UTILS
class FFprobeError(Exception):
    def __init__(self, message):
        self.message = message
        

def ffprobe(filename):
    """
    Returns info about video/audio file using FFprobe.
    Args:
        filename (str): Path to the video file to measure.
    Returns:
        str: decoded FFprobe output (stdout) as one string.
    """
    import subprocess
    command = ['ffprobe', filename]
    process = subprocess.Popen(
        command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)
    try:
        out, err = process.communicate(timeout=10)
    except TimeoutExpired:
        process.kill()
        out, err = process.communicate()

    if err:
        raise FFprobeError(err)
    else:
        if out.splitlines()[-1].find("No such file or directory") != -1:
            raise FileNotFoundError(out.splitlines()[-1])
        else:
            return out
        
        
def get_fps(filename):
    """
    Gets the FPS (frames per second) value of a video using FFprobe.
    Args:
        filename (str): Path to the video file to measure.
    Returns:
        float: The FPS value of the input video file.
    """
    out = ffprobe(filename)
    out_array = out.splitlines()
    video_stream = None
    at_line = -1
    while video_stream == None:
        video_stream = out_array[at_line] if out_array[at_line].find(
            "Video:") != -1 else None
        at_line -= 1
        if at_line < -len(out_array):
            raise NoStreamError(
                "No video stream found. (Is this a video file?)")
    video_stream_array = video_stream.split(',')
    fps = None
    at_chunk = -1
    while fps == None:
        fps = float(video_stream_array[at_chunk].split(
            ' ')[-2]) if video_stream_array[at_chunk].split(' ')[-1] == 'fps' else None
        at_chunk -= 1
        if at_chunk < -len(video_stream_array):
            raise FFprobeError("Could not fetch FPS.")
    return fps
        

def get_framecount(filename, fast=True):
    """
    Returns the number of frames in a video using FFprobe.
    Args:
        filename (str): Path to the video file to measure.
    Returns:
        int: The number of frames in the input video file.
    """
    import subprocess
    command_query_container = 'ffprobe -v error -select_streams v:0 -show_entries stream=nb_frames -of default=nokey=1:noprint_wrappers=1'.split(
        ' ')
    command_query_container.append(filename)
    command_count = 'ffprobe -v error -count_frames -select_streams v:0 -show_entries stream=nb_read_frames -of default=nokey=1:noprint_wrappers=1'.split(
        ' ')
    command_count.append(filename)
    command = command_query_container if fast else command_count

    process = subprocess.Popen(
        command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)
    try:
        out, err = process.communicate(timeout=10)
    except TimeoutExpired:
        process.kill()
        out, err = process.communicate()

    if err:
        raise FFprobeError(err)

    elif out:
        if out.splitlines()[-1].find("No such file or directory") != -1:
            raise FileNotFoundError(out.splitlines()[-1])
        elif out.startswith("N/A"):
            if fast:
                return get_framecount(filename, fast=False)
            else:
                raise FFprobeError(
                    "Could not count frames. (Is this a video file?)")
        else:
            return int(out)

    else:
        if fast:
            return get_framecount(filename, fast=False)
        else:
            raise FFprobeError(
                "Could not count frames. (Is this a video file?)")

In [ ]:
class Motiongram(object):
    
    def __init__(self, path, offset=None, dur=None):
        self.path = path
        self.framecount = get_framecount(path)
        self.fps = get_fps(path) 
        
    def preprocess(video_frame):
        pass

    def motiongram(path):
        pass
    
    
    

In [ ]:
def video2numpy(path, process_func=None, size=None, sr=60.0, offset=None, dur=None, gray=False):
    if not size:
        sizex, sizey = 640, 480  
    else:
        sizex, sizey = size
        
    cap = cv2.VideoCapture(path)
    ret = True

    frames_pro = []
    frames_raw = []
    frameCount = 0
    frameSkipCount = 0
    frames_offset  = int(dur * sr) if offset else int(0)
    frames_to_load = int(dur * sr) if dur else int(1e+10)

    last_frame = np.zeros((sizex, sizey)).T
    
    while ret and (frameSkipCount < frames_offset):
        ret, frame = cap.read()
        frameSkipCount += 1

    while ret and (frameCount < frames_to_load):
        ret, frame = cap.read()
        if ret:
            
            # resize for faster processing
            frame = cv2.resize(frame, (sizex, sizey))
            
            if gray:
                # Use greyscale picture for faster processing
                frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
            else:
                # Convert to standard rgb format (for plotting etc.)
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            
            frames_raw.append(frame)
            
            if process_func:
                # Process the frame (with custom function)
                frame = process_func(frame=frame)
                frames_pro.append(frame)
                
            frameCount += 1
            
    cap.release()
    frames_out_raw = np.stack(frames_raw, axis=0)
    frames_out_pro = np.stack(frames_pro, axis=0) if frames_pro else np.array([])
    
    output_dict = {
        "frames_raw": frames_out_raw,
        "frames_processed": frames_out_pro,
        "duration": frameCount / sr,
        "sampleRate": sr
    }
    
    return frames_out_raw, frames_out_pro 

def findSignificantContour(edgeImg):
    image, contours, hierarchy = cv2.findContours(
        edgeImg,
        cv2.RETR_TREE,
        cv2.CHAIN_APPROX_SIMPLE
    )
    
    # Find level 1 contours
    level1Meta = []
    for contourIndex, tupl in enumerate(hierarchy[0]):
        # Filter the ones without parent
        if tupl[3] == -1:
            tupl = np.insert(tupl.copy(), 0, [contourIndex])
            level1Meta.append(tupl)
    
    # From among them, find the contours with large surface area.
    contoursWithArea = []
    for tupl in level1Meta:
        contourIndex = tupl[0]
        contour = contours[contourIndex]
        area = cv2.contourArea(contour)
        contoursWithArea.append([contour, area, contourIndex])
    
    contoursWithArea.sort(key=lambda meta: meta[1], reverse=True)
    largestContour = contoursWithArea[0][0]
    return largestContour

def extractForeground(frame):
    # Save a copy of the original frame
    frame_orig = np.copy(frame)
    
    # Apply an adaptive threshold
    frame = cv2.adaptiveThreshold(frame, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
                cv2.THRESH_BINARY_INV, 11, 2)
    
    # Filter out salt-and-pepper noise
    frame = cv2.medianBlur(frame, 5)
    
    # Use "close" morphological operation to close the gaps between contours
    frame = cv2.morphologyEx(frame, cv2.MORPH_CLOSE,\
                cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5)));
    
    # Find significant contour
    contour = findSignificantContour(frame)
    
    # Create a mask from the contour
    mask = np.zeros_like(frame)
    cv2.fillPoly(mask, [contour], 255)
    
    # Use "close" morphological operation to close small
    # gaps in the mask
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE,\
                cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5)));
    
    # Mask out the bg
    frame = cv2.bitwise_and(frame_orig, frame_orig, mask=mask)
    
    return frame

def frame_process(*args):
    # Extract foreground (dancer)
    frame = args[0]
    last_frame = args[1]
    
    frameFg = extractForeground(frame)
    
    # Blur the foreground image
    frame = cv2.GaussianBlur(frameFg, (5, 5), 0)
    
    frame_diff = np.abs(frame - last_frame).astype(np.uint8)
    last_frame = np.copy(frame)
    
    # Threshold again
    _, frame = cv2.threshold(frame_diff, 127, 255, cv2.THRESH_BINARY)
    
    # Remove salt-and-pepper noise
    frame = cv2.medianBlur(frame, 5)
    
    # Invert the image (visualisation)
    frame = cv2.bitwise_not(frame)
    
    return frame, last_frame    

# CV TEST SCREEN (Creating MotionGrams)

In [ ]:
relpath  = './../Data/'
filename = 'refined_2M_all_video_url.csv'

all_data = pd.read_csv(relpath + filename, header=None)
data_mask = all_data[0].str.contains("c01")

urls = []
for idx, url in enumerate(all_data[data_mask][0]):
    urls.append(url)

print(len(urls))
test_url = urls[963]
print(test_url)

In [ ]:
mg = Motiongram('./boomwhackers.mp4')
print(mg.framecount, mg.fps)

In [ ]:
# Get the video from test_url:
test_url = './boomwhackers.mp4'
frames_raw, frames_processed = video2numpy(
    test_url, 
    process_func=None,
    sr=24.0, 
    offset=15, 
    dur=60,
    size=(320, 240)
)

print(frames_raw.shape, frames_processed.shape)

In [ ]:
# Make motiongrams
batchSize = 128
batchStepsLo = np.array([i for i in range(0, frames_raw.shape[0], batchSize)])
batchStepsHi = np.minimum(frames_raw.shape[0], batchStepsLo + batchSize)

ft, fy, fx, fc = frames_raw.shape 
frame_diffs = np.zeros((ft-1, fy, fx, fc)).astype(np.float16)
motiongram_x = np.zeros((fx, ft-1, fc)).astype(np.float16)
motiongram_y = np.zeros((fy, ft-1, fc)).astype(np.float16)
vertax = 1
horzax = 2

for lo, hi in zip(batchStepsLo, batchStepsHi):
    #print("lo0: {0}, hi0: {1}".format(lo, hi-1))
    #print("lo1: {0}, hi1: {1}".format(lo+1, hi))
    frames0 = frames_raw[lo:hi-1].astype(np.float16)
    frames1 = frames_raw[lo+1:hi].astype(np.float16)
    fd = np.abs(frames1 - frames0)
    
    frame_diffs[lo:hi-1] = fd
    
    # split into rgb bands
    for c in range(3):
        ch = fd[:, :, :, int(c)]
        chx_mu = np.mean(ch, axis=vertax) #np.mean(ch, axis=vertax)
        chy_mu = np.mean(ch, axis=horzax) #np.mean(ch, axis=horzax)
        motiongram_x[:, lo:hi-1, c] = chx_mu.T
        motiongram_y[:, lo:hi-1, c] = chy_mu.T
    
frame_diffs = frame_diffs.astype(np.uint8)    
motiongram_x = motiongram_x.astype(np.uint8)
motiongram_y = motiongram_y.astype(np.uint8)

#for i in range(3):
#    motiongram_x[:, :, i] = motiongram_x[:, :, i] / np.amax(motiongram_x[:, :, i])
#    motiongram_y[:, :, i] = motiongram_y[:, :, i] / np.amax(motiongram_y[:, :, i])

In [ ]:
print(motiongram_x.shape, motiongram_y.shape)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 10))
plt.imshow(motiongram_y)
plt.show()

#thresh = 25
#frame_diffs[frame_diffs > thresh] = 255
#frame_diffs[frame_diffs <= thresh] = 0
#frame_gray = frame_diffs[:, :, :, 0] * 0.33 + frame_diffs[:, :, :, 1] * 0.33 + frame_diffs[:, :, :, 2] * 0.34


In [ ]:
import librosa
import librosa.display

fig, ax = plt.subplots()
S_dB = librosa.power_to_db(frames_collapsed_x.T, ref=np.max)
img = librosa.display.specshow(S_dB, x_axis='time',
                         y_axis='mel', sr=60,
                         fmax=8000, ax=ax)
fig.colorbar(img, ax=ax, format='%+2.0f dB')
ax.set(title='Mel-frequency spectrogram')

'''
times = librosa.times_like(cent)

fig, ax = plt.subplots()
x_ = np.arange(frames_collapsed_x.shape[0]) / 60.0
y_ = frames_collapsed_x[1] / np.amax(frames_collapsed_x[1])
ax.pcolor(x_, y_, frames_collapsed_x.T, cmap='Spectral_r')
ax.plot(x_, cent.T / np.amax(cent.T), label='Spectral centroid', color='w')
ax.plot(x_, flat.T / np.amax(flat.T), label='Spectral flatness', color='k')
ax.legend(loc='upper right')
'''


In [ ]:
'''
# open webcam video stream
ret = True
cap = cv2.VideoCapture(test_url)

last_frame = np.zeros((640, 480)).T

# Taking a matrix of size 5 as the kernel
kernel = np.ones((5,5), np.uint8)
 
while(ret):
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        continue
    
    # resizing for faster detection
    frame = cv2.resize(frame, (640, 480))
        
    # using a greyscale picture, also for faster detection
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    
    # Save original gray-scale frame
    frame_orig = np.copy(frame)
    
    # Apply an adaptive threshold
    frame = cv2.adaptiveThreshold(frame, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
                cv2.THRESH_BINARY_INV, 11, 2)
    
    frame = cv2.medianBlur(frame, 5)
    
    
    # Use "close" morphological operation to close the gaps between contours
    # https://stackoverflow.com/questions/18339988/implementing-imcloseim-se-in-opencv
    frame = cv2.morphologyEx(frame, cv2.MORPH_CLOSE,\
                cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5)));
    
    # Find contours
    contour = findSignificantContour(frame)
    
    # Draw contours
    #contourImg = np.copy(frame)
    #cv2.drawContours(contourImg, [contour], 0, 127, 2, cv2.LINE_AA, maxLevel=1)
    
    
    # Remove salt-and-pepper noise
    frame = SaltPepperNoise(frame)
    
    # Use "close" morphological operation to close the gaps between contours
    # https://stackoverflow.com/questions/18339988/implementing-imcloseim-se-in-opencv
    frame = cv2.morphologyEx(frame, cv2.MORPH_CLOSE,\
                cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (31, 31)));
    
    # Find contours
    contour = findSignificantContour(frame)
    
    # Draw contours
    #contourImg = np.copy(frame)
    #cv2.drawContours(contourImg, [contour], 0, 127, 2, cv2.LINE_AA, maxLevel=1)
    
    # Create a mask from the contour
    mask = np.zeros_like(frame)
    cv2.fillPoly(mask, [contour], 255)
    
    # Mask out the bg
    frame = cv2.bitwise_and(frame_orig, frame_orig, mask=mask)
    
    frame_diff = np.abs(frame - last_frame).astype(np.uint8)
    last_frame = np.copy(frame)
    
    _, frame = cv2.threshold(frame_diff, 127, 255, cv2.THRESH_BINARY)
    
    frame = SaltPepperNoise(frame)
    
    
    # Create a mask from the contour
    mask = np.zeros_like(frame)
    cv2.fillPoly(mask, [contour], 255)
    
    # Use "close" morphological operation to close the gaps between contours
    # https://stackoverflow.com/questions/18339988/implementing-imcloseim-se-in-opencv
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE,\
                cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5)));
    
    # Mask out the bg
    frame = cv2.bitwise_and(frame_orig, frame_orig, mask=mask)
    
    # Blur the masked image
    frame = cv2.GaussianBlur(frame, (5, 5), 0)
    
    frame_diff = np.abs(frame - last_frame).astype(np.uint8)
    last_frame = np.copy(frame)
    
    # Threshold again
    _, frame = cv2.threshold(frame_diff, 127, 255, cv2.THRESH_BINARY)
    
    # Median filter again
    frame = cv2.medianBlur(frame, 5)
    
    # Invert
    frame = cv2.bitwise_not(frame)
    
    
    # calculate sure foreground area by dilating the mask
    #mapFg = cv2.erode(mask, np.ones((5, 5), np.uint8), iterations=10)    

    # mark inital mask as "probably background"
    # and mapFg as sure foreground
    trimap = np.copy(mask)
    trimap[mask  == 0]   = cv2.GC_BGD
    trimap[mask  == 255] = cv2.GC_PR_BGD
    trimap[mapFg == 255] = cv2.GC_FGD
    
    # visualize trimap
    trimap_print = np.copy(trimap)
    trimap_print[trimap_print == cv2.GC_PR_BGD] = 128
    trimap_print[trimap_print == cv2.GC_FGD] = 255
    

    # Display the resulting frame
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()

# finally, close the window
cv2.destroyAllWindows()
cv2.waitKey(10)
'''

In [ ]:
h = "http.//www.something.com/path1/path2.wav"
idx = h.rindex('/')
print(h[idx+1:])